# Calculate ice flux from bed topo, elevation, and velocity time series

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import numpy as np
import datetime
from tqdm.auto import tqdm
import xesmf as xe
from matplotlib.patches import Rectangle

In [ ]:
# -----Define path to data
data_path = '/Users/raineyaberle/Research/PhD/Hubbard/'
figures_out_path = os.path.join(data_path, 'figures')

# -----Load bed topography
bed_fn = os.path.join(data_path, 'bed_topo', 'hubbard_mc_idw_bedElev.tif')
bed = rxr.open_rasterio(bed_fn)
bed = bed.rio.reproject('EPSG:3338')
bed = xr.where((bed > 3e38) | (bed < -3e38), np.nan, bed)
bed = bed.rio.write_crs('EPSG:3338')

# -----Load surface elevation file names
h_fns = sorted(glob.glob(os.path.join(data_path, 'surface_elevation', 'coregistered', '*.tif')))
[os.path.basename(x) for x in h_fns]

In [ ]:
# -----Define function for reprojecting and regridding to bed topography grid
def reproject_regrid(da, ref_da):
    da = da.rio.reproject(ref_da.rio.crs)
    da_interp = da.interp(x=ref_da.x.data, y=ref_da.y.data)
    return da_interp

# -----Iterate over surface elevation file names
xmin, xmax = 795, 810
ymin, ymax = 1201, 1215.5
H_created = False
for i, h_fn in enumerate(h_fns):
    # Grab date from file name
    if 'SETSM' in os.path.basename(h_fn):
        date = os.path.basename(h_fn).split('_')[3]
        dt = datetime.datetime(int(date[0:4]), int(date[4:6]), int(date[6:8]))
        date = date[0:4] + '-' + date[4:6] + '-' + date[6:8]
    elif 'ILAK' in h_fn:
        year = os.path.basename(h_fn).split('_')[1]
        julian_day = os.path.basename(h_fn).split('_')[2]
        dt = datetime.datetime.strptime(year+julian_day, '%Y%j')
        date = str(dt)
    elif ('Hubbard.' in h_fn) or ('Valerie.' in h_fn):
        year = os.path.basename(h_fn).split('.')[1]
        julian_day = os.path.basename(h_fn).split('.')[2][0:3]
        dt = datetime.datetime.strptime(year+julian_day, '%Y%j')
        date = str(dt)
    print(dt)
    # Skip if before the first velocity date
    v_time = datetime.datetime(int(str(v.time.data[0])[0:4]), 
                                   int(str(v.time.data[0])[5:7]), 
                                   int(str(v.time.data[0])[8:9]))
    if dt < v_time:
        continue

    # Load surface elevation
    h = rxr.open_rasterio(h_fn)
    
    # Reproject and interpolate all datasets to bed elevation coordinates
    h_interp = reproject_regrid(h, bed)

    # Remove no data values
    h_interp = xr.where(h_interp==h_interp._FillValue, np.nan, h_interp) 
    
    # Calculate thickness
    H_date = h_interp - bed
    # convert to dataset
    H_date = H_date.to_dataset('band').rename({1: 'thickness'})
    # add time dimension
    H_date = H_date.expand_dims({'time': [dt]})

    # Check for data coverage
    if (np.isnan(H_date.thickness.data[0])).all():
        continue

    # Save in dataset
    if H_created:
        H = xr.concat([H, H_date], dim='time')
    else:
        H = H_date.copy()
        H_created = True

    # Plot
    # fig, ax = plt.subplots()
    # H_im = ax.imshow(H_date.thickness.data[0], cmap='plasma', clim=(0, 1000),
    #                   extent=(np.min(H_date.x.data)/1e3, np.max(H_date.x.data)/1e3, 
    #                           np.min(H_date.y.data)/1e3, np.max(H_date.y.data)/1e3))
    # ax.set_title(date)
    # ax.set_xlabel('Easting [km]')
    # ax.set_ylabel('Northing [km]')
    # ax.set_xlim(xmin, xmax)
    # ax.set_ylim(ymin, ymax)
    # fig.colorbar(H_im, ax=ax, label='Thickness [m]')
    # plt.show()

    # # Save figure
    # fig_fn = os.path.join(figures_out_path, 'thickness_' + date[0:10] + '.png')
    # fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
    # print('figure saved to file:', fig_fn)

# -----Save thickness to file
date_start = str(H.time.data[0])[0:4]
date_end = str(H.time.data[-1])[0:4]
H_fn = os.path.join(data_path, 'ice_thickness', f'ice_thickness_{date_start}-{date_end}.nc')
H = H.rio.write_crs('EPSG:3338')
H.to_netcdf(H_fn)
print('Ice flux saved to file:', H_fn)

## Calculate thickness anomalies over time

In [ ]:
# Calculate median thickness everywhere
H_median = H.median(dim='time')
# plot median thickness
fig, ax = plt.subplots()
H_im = ax.imshow(H_median.thickness.data, cmap='terrain',
               extent=(np.min(H_median.x.data)/1e3, np.max(H_median.x.data)/1e3,
                       np.min(H_median.y.data)/1e3, np.max(H_median.y.data)/1e3))
ax.set_xlabel('Easting [km]')
ax.set_ylabel('Northing [km]')
ax.set_title('Median thickness')
fig.colorbar(H_im, ax=ax, shrink=0.8, label='Thickness [m]')
plt.show()

In [ ]:
# Calculate difference from median
H_diff = H - H_median
H_diff = H_diff.rename({'thickness':'thickness_diff_from_median'})

# Save to file
H_diff_fn = H_fn.replace('ice_thickness_', 'ice_thickness_anomaly_')
H_diff.to_netcdf(H_diff_fn)
print('Thickness anomalies saved to file:', H_diff_fn)

# Iterate over time stamps to plot
for i in range(0, len(H.time.data)):
    # subset thickness anomaly to date
    H_diff_date = H_diff.isel(time=i)
    # plot
    fig, ax = plt.subplots()
    H_diff_im = ax.imshow(H_diff_date.thickness_diff_from_median.data, cmap='coolwarm_r', clim=(-20, 20),
                          extent=(np.min(H_diff_date.x.data)/1e3, np.max(H_diff_date.x.data)/1e3,
                                  np.min(H_diff_date.y.data)/1e3, np.max(H_diff_date.y.data)/1e3))
    ax.set_xlabel('Easting [km]')
    ax.set_ylabel('Northing [km]')
    ax.set_title(str(H_diff.time.data[i])[0:10])
    fig.colorbar(H_diff_im, ax=ax, shrink=0.8, label='Thickness anomaly [m]')
    plt.show()

    # Save figure
    fig_fn = os.path.join(figures_out_path, 'thickness_anomaly_' + str(H.time.data[i])[0:10] + '.png')
    fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
    print('figure saved to file:', fig_fn)

In [ ]:
# Make a gif
import subprocess

# Construct command
cmd = f"convert -delay 200 {os.path.join(figures_out_path, 'thickness_anomaly*.png')} {os.path.join(figures_out_path, 'thickness_anomalies.gif')}"

# Construct command
# cmd = f'ffmpeg -y -i {os.path.join(figures_out_path, 'frames', '*.png')} {os.path.join(figures_out_path, 'thickness_anomalies.mp4')}'

# Run command
output = subprocess.run(cmd, shell=True, capture_output=True)
print(output)

## Calculate thickness change at points

In [ ]:
# Load IFSAR for reference plot
ifsar_fn = os.path.join(data_path, 'surface_elevation', 'ifsar_hubbardDEM.tif')
ifsar = xr.open_dataset(ifsar_fn)

In [ ]:
point1 = (808e3, 1209e3)
point2 = (802e3, 1204e3)

H_diff_point1 = H_diff.sel(x=point1[0], y=point1[1], method='nearest').thickness_diff_from_median.data
H_diff_point2 = H_diff.sel(x=point2[0], y=point2[1], method='nearest').thickness_diff_from_median.data

# Plot
plt.rcParams.update({'font.sans-serif': 'Arial', 'font.size': 12})
fig, ax = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'width_ratios': [1,4]})
ax[0].imshow(ifsar.band_data.data[0], cmap='Greys_r', clim=(0,1200), 
             extent=(np.min(ifsar.x.data), np.max(ifsar.x.data),
                     np.min(ifsar.y.data), np.max(ifsar.y.data)))
ax[0].plot(*point1, 'o', markerfacecolor='#41b6c4', markeredgecolor='w', markersize=10)
ax[0].plot(*point2, 'o', markerfacecolor='#081d58', markeredgecolor='w', markersize=10)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_xlim((794e3, 810e3))
ax[0].set_ylim(1199e3, 1217e3)
ax[0].set_title('IFSAR, point locations')
ax[1].plot(H.time.data.astype('datetime64'), H_diff_point1, '.', color='#41b6c4', markersize=10)
ax[1].plot(H.time.data.astype('datetime64'), H_diff_point2, '.', color='#081d58', markersize=10)
ax[1].set_ylim(-15, 15)
ax[1].set_ylabel('Thickness anomaly [m]')
ax[1].grid()
# plot rectangles over summer/winter months
years = H.time.data.astype('datetime64[Y]')
for year in years:
    summer_start = np.datetime64(str(year) + '-06-22')
    summer_end = np.datetime64(str(year) + '-09-22')
    spatch = ax[1].add_patch(Rectangle((summer_start, -20), width=(summer_end-summer_start), height=40, facecolor='#fff5eb', edgecolor='None'))
    winter_start = np.datetime64(str(year) + '-12-22')
    winter_end = np.datetime64(str(year+1) + '-03-22')
    wpatch = ax[1].add_patch(Rectangle((winter_start, -20), width=(winter_end-winter_start), height=40, facecolor='#f0f0f0', edgecolor='None'))

ax[1].legend([spatch, wpatch], ['summer', 'winter'], loc='lower right', framealpha=1)
plt.show()

fig.savefig(os.path.join(figures_out_path, 'thickness_anomaly_points.png'), dpi=250, bbox_inches='tight')